### 中醫醫案症狀詞抽取

- model: CRF
- 資料集: 舌象
- 類別標註: OS
- 詞位標注: 無
- 拆句: 以句點拆分
- 斷詞: 沒有做
- 詞性標註: 使用中科院的
- 引用辭典: 沒有
- 標點: 沒去

In [1]:
import json
import nltk
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [2]:
def getTagListWithString(str):
    list1 = []
    str = str.replace('，',' ')
    str = str.replace('、',' ')
    str = str.replace(',',' ')
    str_list = str.split(" ")
    
    for s in str_list:
        s = s.strip()
        if len(s) > 0:
            list1.append(s)
    
    #list2 = list(filter(None, second_list))

    return list1
    

In [3]:
def getLabelWithWordIndex(i,ne_index_list):
    for (ne_idx,ne_len) in ne_index_list: 
        if i >= ne_idx and i <= (ne_idx + ne_len - 1):
            return "S"
    
    return "O"

In [4]:
filename = "case_summary.csv"
with open(filename) as f:
    lines = f.readlines()

docs = []
c = 0
t = 0
for idx,line in enumerate(lines):
    cols = line.strip().split(",")
    
    #原文
    case_content = cols[2]
    case_content = case_content.replace(" ","")
    case_content = case_content.replace('#',',')
    sentences = case_content.split("。")
    
    #摘要
    summary_str = cols[3].strip()
    summary_str = summary_str.strip('"')
    summary_str = summary_str.replace('""','"')
    summary_str = summary_str.replace('#',',')
    
    if idx == 0 or len(summary_str) == 0 :
        continue
        
    summary_json = json.loads(summary_str)
        
    keys = summary_json.keys()
    if "tongue" in keys:
        main = summary_json["tongue"]
        main_list = getTagListWithString(main)
        #print(main_list)
    
    t += len(main_list)

    for sent in sentences:
        ne_index_list = []
        for ne in main_list:
            ne_idx = sent.find(ne)
            if ne_idx != -1 :
                ne_index_list.append((ne_idx,len(ne)))

        texts = []
        for i,w in enumerate(sent):
            label = getLabelWithWordIndex(i,ne_index_list)
            texts.append((w,label))

        docs.append(texts)

print(len(docs))

48901


In [5]:
### 計算平均句長
total_wc = 0
for doc in docs:
    total_wc += len(doc)
    
ave = total_wc / 48901
print(ave)

32.537698615570235


In [6]:
import pynlpir  # 引入依赖包
pynlpir.open()  # 打开分词器

data = []

for i, doc in enumerate(docs):
    tokens = [t for t, label in doc]
    
    tagged = []
    for tk in tokens:
        tg = pynlpir.segment(tk)
        tagged.append(tg[0])
    
    data.append([(w, pos, label) for (w, label), (word, pos) in zip(doc, tagged)])

part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gjtgj'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gjtgj'
part of speech not recognized: 'gms'
part of speech not recognized: 'gm

In [7]:
def word2features(doc, i):
    word = doc[i][0]
    postag = doc[i][1]
    
    if postag is None:
        postag = 'none'

    # Common features for all words
    features = [
        'bias',
        'word=' + word,
        'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag
    ]
        
    # Features for words that are not at the beginning of a document
    if i > 0:
        word1 = doc[i-1][0]
        postag1 = doc[i-1][1]
        
        if postag1 is None:
            postag1 = 'none'
        
        features.extend([
            '-1:word=' + word1,
            '-1:word.isdigit=%s' % word1.isdigit(),
            '-1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'beginning of a document'
        features.append('BOS')

    # Features for words that are not at the end of a document
    if i < len(doc)-1:
        word1 = doc[i+1][0]
        postag1 = doc[i+1][1]
        
        if postag1 is None:
            postag1 = 'none'
        
        features.extend([
            '+1:word=' + word1,
            '+1:word.isdigit=%s' % word1.isdigit(),
            '+1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'end of a document'
        features.append('EOS')

    return features

In [8]:
# A function for extracting features in documents
def extract_features(doc):
    return [word2features(doc, i) for i in range(len(doc))]

# A function fo generating the list of labels for each document
def get_labels(doc):
    return [label for (token, postag, label) in doc]

X = [extract_features(doc) for doc in data]
y = [get_labels(doc) for doc in data]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [9]:
print(len(X_train))
print(len(X_test))

39120
9781


In [10]:
import pycrfsuite
trainer = pycrfsuite.Trainer(verbose=True)

# Submit training data to the trainer
for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

# Set the parameters of the model
trainer.set_params({
    # coefficient for L1 penalty
    'c1': 0.1,

    # coefficient for L2 penalty
    'c2': 0.01,  

    # maximum number of iterations
    'max_iterations': 200,

    # whether to include transitions that
    # are possible, but not observed
    'feature.possible_transitions': True
})

# Provide a file name as a parameter to the train function, such that
# the model will be saved to the file when training is finished
trainer.train('crf.model')

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 11946
Seconds required: 1.646

L-BFGS optimization
c1: 0.100000
c2: 0.010000
num_memories: 6
max_iterations: 200
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 83901.056904
Feature norm: 1.000000
Error norm: 106121.899903
Active features: 11942
Line search trials: 1
Line search step: 0.000000
Seconds required for this iteration: 0.463

***** Iteration #2 *****
Loss: 78608.056265
Feature norm: 1.050152
Error norm: 83252.995843
Active features: 11854
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.237

***** Iteration #3 *****
Loss: 66784.119950
Feature norm: 1.232090
Error norm: 32601.728442
Active features: 10287
Line search trials: 1
Line search step: 1.000000
Seconds required fo

***** Iteration #46 *****
Loss: 6094.966492
Feature norm: 23.768411
Error norm: 4643.149412
Active features: 3265
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.238

***** Iteration #47 *****
Loss: 6006.287160
Feature norm: 24.850390
Error norm: 423.269174
Active features: 3251
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.234

***** Iteration #48 *****
Loss: 5958.391952
Feature norm: 26.245237
Error norm: 839.352361
Active features: 3204
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.239

***** Iteration #49 *****
Loss: 5894.879065
Feature norm: 30.926530
Error norm: 3016.704315
Active features: 3105
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.242

***** Iteration #50 *****
Loss: 5847.604860
Feature norm: 31.850419
Error norm: 1231.628823
Active features: 3106
Line search trials: 1
Line search step: 1.000000
Seconds req

***** Iteration #89 *****
Loss: 5348.712448
Feature norm: 45.456349
Error norm: 861.784183
Active features: 2509
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.236

***** Iteration #90 *****
Loss: 5344.790668
Feature norm: 45.471275
Error norm: 363.119796
Active features: 2498
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 0.467

***** Iteration #91 *****
Loss: 5343.159331
Feature norm: 45.476459
Error norm: 1063.244773
Active features: 2490
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.238

***** Iteration #92 *****
Loss: 5339.517532
Feature norm: 45.515576
Error norm: 632.022878
Active features: 2485
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.235

***** Iteration #93 *****
Loss: 5337.227903
Feature norm: 45.517560
Error norm: 654.447811
Active features: 2483
Line search trials: 1
Line search step: 1.000000
Seconds requi

***** Iteration #130 *****
Loss: 5250.154525
Feature norm: 45.046251
Error norm: 310.731177
Active features: 2160
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.248

***** Iteration #131 *****
Loss: 5248.900878
Feature norm: 45.018713
Error norm: 326.971152
Active features: 2158
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.234

***** Iteration #132 *****
Loss: 5247.521160
Feature norm: 44.991010
Error norm: 344.646136
Active features: 2139
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.234

***** Iteration #133 *****
Loss: 5245.797208
Feature norm: 44.938235
Error norm: 373.244079
Active features: 2137
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.233

***** Iteration #134 *****
Loss: 5244.360085
Feature norm: 44.879300
Error norm: 543.578368
Active features: 2128
Line search trials: 1
Line search step: 1.000000
Seconds r

***** Iteration #175 *****
Loss: 5193.629833
Feature norm: 45.653693
Error norm: 183.469488
Active features: 1967
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.249

***** Iteration #176 *****
Loss: 5193.571906
Feature norm: 45.673911
Error norm: 576.950073
Active features: 1968
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.242

***** Iteration #177 *****
Loss: 5192.608475
Feature norm: 45.675548
Error norm: 209.214495
Active features: 1965
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.239

***** Iteration #178 *****
Loss: 5192.476211
Feature norm: 45.689299
Error norm: 509.111654
Active features: 1958
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.253

***** Iteration #179 *****
Loss: 5191.579101
Feature norm: 45.689393
Error norm: 156.258503
Active features: 1954
Line search trials: 1
Line search step: 1.000000
Seconds r

In [11]:
tagger = pycrfsuite.Tagger()
tagger.open('crf.model')

y_pred_train = [tagger.tag(xseq) for xseq in X_train]
y_pred_test = [tagger.tag(xseq) for xseq in X_test]

In [12]:
# Create a mapping of labels to indices
labels = {"O": 0,"S": 1}

# training data F1-score
predictions_train = np.array([labels[tag] for row in y_pred_train for tag in row])
truths_train = np.array([labels[tag] for row in y_train for tag in row])
print("F1 score for training set:\n")
print(classification_report(
    truths_train, predictions_train,
    target_names=["O", "S"]))

print("\n")

# test data F1-score
predictions_test = np.array([labels[tag] for row in y_pred_test for tag in row])
truths_test = np.array([labels[tag] for row in y_test for tag in row])
print("F1 score for test set:\n")
print(classification_report(
    truths_test, predictions_test,
    target_names=["O", "S"]))

F1 score for training set:

             precision    recall  f1-score   support

          O       1.00      1.00      1.00   1259654
          S       0.67      0.86      0.75     12942

avg / total       1.00      0.99      0.99   1272596



F1 score for test set:

             precision    recall  f1-score   support

          O       1.00      1.00      1.00    315324
          S       0.65      0.84      0.74      3206

avg / total       0.99      0.99      0.99    318530

